In [ ]:
import arviz as az
import numpy as np
import pandas as pd
import scipy as sp
import altair as alt

import pymc as pm
import pytensor.tensor as pt
import xarray as xr

from utils import *

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


# Data Loading

In [ ]:
data_file = "./data/md.csv"
data = pd.read_csv(data_file,index_col=0)
data.head()

In [ ]:
# Convert the ordinal columns to ordered categories
cats = ['I have no confidence', "I don't have much confidence", 'No answer', 'Some confidence', 'A lot of confidence']
for c in data.columns[9:]:
    data[c] = pd.Categorical(data[c], cats, ordered=True)

# Convert other columns to categorical
for c in data.columns[:9]:
    data[c] = pd.Categorical(data[c])#,list(data[c].unique()))

data.head()

# Basic Linear model

## Define `coords`

In [4]:
# Transform the categorical inputs to integers
age_idx, ages = data["age_group2"].factorize(sort=True)
education_idx, educations = data["education"].factorize(sort=True)
gender_idx, genders = data["gender"].factorize(sort=True)
ethnicity_idx, ethnicities = data["ethnicity"].factorize(sort=True)
unit_idx, units = data["county"].factorize(sort=True)
ur_idx, ur_responses = data["ur"].factorize(sort=True) # Urban/Rural

COORDS = {
        "age": ages,
        "education": educations,
        "gender": genders,
        "ethnicity": ethnicities,
        "unit": units,
        "ur": ur_responses,
        
        # Create a dimension for observations
        'obs_idx': list(range(len(data)))
}

# Basic Model

In [ ]:
# What are we interested in predicting?
dependent_col = 'trust2_EU'

with pm.Model(coords=COORDS) as model:

    # --------------------------------------------------------
    #                   Intercepts
    # --------------------------------------------------------

    # intercept (i.e national) latent popularity for each response
    intercept = pm.Normal("intercept", sigma=1.5)

    # We collect all effects here. It will be of dimension (obs_idx, response)
    mu = intercept

    # --------------------------------------------------------
    #                   DATA COMPONENTS
    # --------------------------------------------------------
    sigma = 0.5

    # Declare inputs as Data so we can change them in post-stratification
    age_id = pm.Data("age_id", age_idx, dims="obs_idx")
    a_age = pm.Normal("a_age", sigma=sigma, dims="age")
    mu += a_age[age_id]

    education_id = pm.Data("education_id", education_idx, dims="obs_idx")
    a_edu = pm.Normal("a_edu", sigma=sigma, dims="education")
    mu += a_edu[education_id]

    gender_id = pm.Data("gender_id", gender_idx, dims="obs_idx")
    a_gender = pm.Normal("a_gender", sigma=sigma, dims="gender")
    mu += a_gender[gender_id]

    ethnicity_id = pm.Data("ethnicity_id", ethnicity_idx, dims="obs_idx")
    a_eth = pm.Normal("a_eth", sigma=sigma, dims="ethnicity")
    mu += a_eth[ethnicity_id]
    
    unit_id = pm.Data("unit_id", unit_idx, dims="obs_idx")
    a_unit = pm.Normal("a_unit", sigma=sigma, dims="unit")
    mu += a_unit[unit_id]

    ur_id = pm.Data("ur_id", ur_idx, dims="obs_idx")
    a_ur = pm.Normal("a_ur", sigma=sigma, dims="ur")
    mu += a_ur[ur_id]

    # We can also add interaction terms:
    a_age_edu = pm.Normal(
        "a_age_edu",
        sigma=0.5*sigma,
        dims=("age", "education"),
    )
    mu += a_age_edu[age_id, education_id]

    a_gender_eth = pm.Normal(
        "a_gender_eth",
        sigma=0.5*sigma,
        dims=("gender", "ethnicity"),
    )
    mu += a_gender_eth[gender_id, ethnicity_id]


    # --------------------------------------------------------
    #                   OBSERVATION MODEL
    # -------------------------------------------------------- 

    multinomial = True

    if multinomial == False: # Simple model, but hard to post-stratify

        resp_idx, resp_cat = data[dependent_col].factorize(sort=True)
        pm.Binomial("y", p=p, N=len(resp_cat)-1, observed=resp_idx)

    else: # Effectively the same model as above, but built for post-stratification

        # Create a mutable N and set it to 1 for now as we have one observation per person
        # This will be changed to the size of the census segment in post-stratification
        N = pm.Data("N", np.ones(len(data)), dims="obs_idx")

        # Convert the dependent variable to a dummy matrix
        obs = pd.get_dummies(data[dependent_col]).astype(int)

        # Add a coord for it as well
        model.add_coord('response',list(obs.columns))

        # Compute the p for each respondent
        p = pm.math.invlogit(mu)

        # This is hand-made, code in utils.py
        # It's the multivariate distribution of answers
        # IFF everyone answered according to a binomial distribution
        MvBinomial("y", p=p, N=len(obs.columns)-1, 
            n=N, observed=obs.values, dims=("obs_idx", "response"))

pm.model_to_graphviz(model)

In [ ]:
with model:
    idata = pm.sample(nuts_sampler='nutpie', idata_kwargs={"log_likelihood": True}) 

In [ ]:
# Save the fit
idata.to_netcdf('ord_mrp.nc')

In [ ]:
az.plot_trace(idata, var_names=["~_offset","~_probs"], filter_vars="regex");

# Post-stratification

### Process census data

In [ ]:
cdata = pd.read_csv("./data/md_census.csv")
cdata.head()

In [ ]:
# Drop 0 values
cdata = cdata[cdata.N > 0]
print(len(cdata))

# Drop the <15 category
cdata = cdata[cdata.age_group != '<15']

# Map the age groups to the groups used in the poll
cdata['age_group2'] = cdata['age_group'].replace({'15-19': '15-19',
 '20-24': '20-29',
 '25-29': '20-29',
 '30-34': '30-39',
 '35-39': '30-39',
 '40-44': '40-49',
 '45-49': '40-49',
 '50-54': '50-59',
 '55-59': '50-59',
 '60-64': '60+',
 '65-69': '60+',
 '70+': '60+'})

# Merge over all the columns that are present in both dataframes
shared_cols = [c for c in cdata.columns if c in data.columns]
cdata = cdata.groupby(shared_cols)['N'].sum().reset_index()

cdata.head()

In [9]:
# Check that the columns that are present in both have matching category values
for c in cdata.columns[:-1]:
    s1, s2 = set(data[c].unique()), set(cdata[c].unique())
    if s1 != s2:
        print(c, s1-s2, s2-s1)


### Define new dimensions and coordinates

In [10]:

# I'm surprised pandas does not have this function but I could not find it. 
def factorize_w_codes(s, codes):
    res = s.replace(dict(zip(codes,range(len(codes)))))
    if not s.isin(codes).all(): # Throw an exception if all values were not replaced
        vals = set(s) - set(codes)
        raise Exception(f'Codes for {s.name} do not match all values: {vals}')
    return res.to_numpy()

In [ ]:
poststrat_age_idx = factorize_w_codes(cdata["age_group2"],ages)
poststrat_education_idx = factorize_w_codes(cdata["education"],educations)
poststrat_gender_idx = factorize_w_codes(cdata["gender"],genders)
poststrat_ethnicity_idx = factorize_w_codes(cdata["ethnicity"],ethnicities)
poststrat_units_idx = factorize_w_codes(cdata["county"],units)
poststrat_ur_idx = factorize_w_codes(cdata["ur"],ur_responses)

### Condition model on new data

In [ ]:
new_data = {
            "age_id": poststrat_age_idx,
            "education_id": poststrat_education_idx,
            "gender_id": poststrat_gender_idx,
            "ethnicity_id": poststrat_ethnicity_idx,
            "unit_id": poststrat_units_idx,
            "ur_id":poststrat_ur_idx,
            # "obs": np.zeros(
            #     (len(cdata.index), len(y_cols), len(y_options) ),
            #     dtype=int,
            # ),  # just a placeholder for new data size
            "N": cdata["N"].to_numpy(),
        }

with model:
    pm.set_data(
        coords={
            "obs_idx": cdata.index,
        },
        new_data=new_data
    )
    
    print("Sampling posterior predictive on census data")
    # Reduce the sample to just 100 draws from 2 chains each
    idata_s = idata.sel(draw=slice(0,500),chain=[0,1])
    idata_ps = pm.sample_posterior_predictive(
        idata_s,
        log_likelihood=True,
        predictions=True
    )

### Create the "Synthetic Population"

In [19]:
pred = idata_ps.predictions.y

# This needed some technical work to not explode memory usage
df = pd.DataFrame(
    pred.values.reshape(-1), dtype=pd.Int16Dtype(),
    columns=['N']
).assign(
    chain=np.repeat(pred.chain.values, len(pred.draw) * len(pred.obs_idx)*len(pred.response)).astype('int8'),
    draw=np.tile(np.repeat(pred.draw.values, len(pred.obs_idx)*len(pred.response)), len(pred.chain)).astype('int16'),
    obs_idx=np.tile(pred.obs_idx.values, len(pred.chain) * len(pred.draw)*len(pred.response)).astype('int16'),
    response=np.tile(np.arange(len(pred.response)), len(pred.chain) * len(pred.draw)*len(pred.obs_idx)).astype('int8')
)
df['adraw'] = df['draw'] + df['chain'] * len(pred.draw)
#df.memory_usage(deep=True)

# Sample a "synthetic population" of 1M rows by sampling with replacement with weights based on N
df = df.sample(weights='N',n=1000000,replace=True).drop(columns=['N'])
df['response'] = pd.Categorical(pred.response.values[df['response']],categories=pred.response.values,ordered=True)

# Merge with the census data on obs_idx
df = df.merge(cdata,left_on='obs_idx',right_index=True)
df = df.drop(columns=['obs_idx','N','chain','draw'])


# Explore synthetic data

In [ ]:
# The synthetic population is now ready to be used for analysis
df.head()

In [ ]:
from plotnine import *

# Draw topline distribution
agg = (df.groupby(['response','adraw']).size()/df.groupby('adraw').size()).reset_index(name='p')
(ggplot(agg, aes(x='response', y='p')) +
    geom_boxplot() +
    theme(axis_text_x=element_text(angle=45, hjust=1)) +
    labs(x='Response', y='Proportion', title='Distribution of Response Proportions')
)

In [ ]:
import altair as alt

alt.Chart(data).mark_bar().encode(
    x=dependent_col,
    y='count()',
).properties(
    width=600,
    height=400
)

In [ ]:
# Draw young urban female highly educated only
fquery = "gender == 'Female' and age_group2.isin(['15-19','20-29']) and education == 'Higher education'"

print(f"N in original data: {len(data.query(fquery))}")
print(f"N in synthetic population: {len(df.query(fquery))}")

f_df = df.query(fquery)
agg = (f_df.groupby(['response','adraw']).size()/f_df.groupby('adraw').size()).reset_index(name='p')
(ggplot(agg, aes(x='response', y='p')) +
    geom_boxplot() +
    theme(axis_text_x=element_text(angle=45, hjust=1)) +
    labs(x='Response', y='Proportion', title='Distribution of Response Proportions')
)

# Other likelihoods for Order

In [ ]:
# What are we interested in predicting?
dependent_col = 'trust2_EU'
model_type = 'stereotype'

with pm.Model(coords=COORDS) as model:

    # --------------------------------------------------------
    #                   Intercepts
    # --------------------------------------------------------

    # intercept (i.e national) latent popularity for each response
    intercept = pm.Normal("intercept", sigma=1.5)

    # We collect all effects here. It will be of dimension (obs_idx, response)
    mu = intercept

    # --------------------------------------------------------
    #                   DATA COMPONENTS
    # --------------------------------------------------------
    sigma = 0.5

    # Declare inputs as Data so we can change them in post-stratification
    age_id = pm.Data("age_id", age_idx, dims="obs_idx")
    a_age = pm.Normal("a_age", sigma=sigma, dims="age")
    mu += a_age[age_id]

    education_id = pm.Data("education_id", education_idx, dims="obs_idx")
    a_edu = pm.Normal("a_edu", sigma=sigma, dims="education")
    mu += a_edu[education_id]

    gender_id = pm.Data("gender_id", gender_idx, dims="obs_idx")
    a_gender = pm.Normal("a_gender", sigma=sigma, dims="gender")
    mu += a_gender[gender_id]

    ethnicity_id = pm.Data("ethnicity_id", ethnicity_idx, dims="obs_idx")
    a_eth = pm.Normal("a_eth", sigma=sigma, dims="ethnicity")
    mu += a_eth[ethnicity_id]
    
    unit_id = pm.Data("unit_id", unit_idx, dims="obs_idx")
    a_unit = pm.Normal("a_unit", sigma=sigma, dims="unit")
    mu += a_unit[unit_id]

    ur_id = pm.Data("ur_id", ur_idx, dims="obs_idx")
    a_ur = pm.Normal("a_ur", sigma=sigma, dims="ur")
    mu += a_ur[ur_id]

    # We can also add interaction terms:
    a_age_edu = pm.Normal(
        "a_age_edu",
        sigma=0.5*sigma,
        dims=("age", "education"),
    )
    mu += a_age_edu[age_id, education_id]

    a_gender_eth = pm.Normal(
        "a_gender_eth",
        sigma=0.5*sigma,
        dims=("gender", "ethnicity"),
    )
    mu += a_gender_eth[gender_id, ethnicity_id]
    pm.Deterministic("mu", mu, dims="obs_idx")

    # --------------------------------------------------------
    #                   OBSERVATION MODEL
    # -------------------------------------------------------- 

    # Create a mutable N and set it to 1 for now as we have one observation per person
    # This will be changed to the size of the census segment in post-stratification
    N = pm.Data("N", np.ones(len(data)), dims="obs_idx")

    # Convert the dependent variable to a dummy matrix
    obs = pd.get_dummies(data[dependent_col]).astype(int)
    model.add_coord('response',list(obs.columns))

    # Binomial observation model
    if model_type == 'binomial':
        p = pm.math.invlogit(mu)
        MvBinomial("y", p=p, N=len(obs.columns)-1, 
            n=N, observed=obs.values, dims=("obs_idx", "response"))
            
    # Ordered Multinomial observation model (Logit-based)
    elif model_type == 'ordered-logit':
        # Model for cutpoints
        cp_scale = pm.HalfNormal(f"cutpoint_sd", sigma=2)
        cp_len = len(obs.columns)-1
        
        cutpoints = pm.ZeroSumNormal(
            "cutpoints",
            sigma=cp_scale,
            transform=pm.distributions.transforms.ordered,
            initval=np.linspace(-1,1,cp_len),
            shape=cp_len
        )

        # Ordered Multinomial observation model (Logit-based)
        pm.OrderedMultinomial(
            "y",
            eta=mu,
            cutpoints=cutpoints,
            n=N,
            observed=obs.values,
            dims=("obs_idx", "response"),
        )

    # Stereotype regression
    elif model_type == 'stereotype':

        # Generate phi values: first is 0, last is 1 and the inbetween is increasing
        n_ordered = len(obs.columns)
        phi_delta = pm.Dirichlet(f'phi_diffs', [1.0]*(n_ordered-1) )
        phi = pt.concatenate([[0], pt.cumsum(phi_delta)])

        # Create baseline values
        b = (pm.Normal("stereotype_intercept", size=n_ordered, 
                                transform=pm.distributions.transforms.ZeroSumTransform([-1])))

        # Log_odds = baseline + phi*mu
        log_odds = b[None,:] + phi[None,:]*mu[:,None]
        
        # Run them through softmax
        probs = pm.math.softmax(log_odds, axis=-1)

        pm.Multinomial("y", p=probs, n=N, observed=obs.values, dims=("obs_idx", "response"))

# Do the same thing as above for post-stratification. Code in utils.py
df_om, idata_om = poststratify(model,cdata,new_data)

# Hierarchical block

In [11]:
# A hierarchical building block for the model    
def hierarchical_zsn(name: str, dim: str, sigma: float = 0.2):

    scale = pm.HalfNormal(f"{name}_sd", sigma=sigma)
    offset = pm.ZeroSumNormal(f"{name}_offset", dims=dim)
    
    # Effect = scale * offset
    return pm.Deterministic(f"{name}", scale * offset, dims=(dim))

# Improved model

In [ ]:
# What are we interested in predicting?
dependent_col = 'trust2_EU'
model_type = 'binomial'

with pm.Model(coords=COORDS) as model:

    # --------------------------------------------------------
    #                   Intercepts
    # --------------------------------------------------------

    # intercept (i.e national) latent popularity for each response
    intercept = pm.Normal("intercept", sigma=2)

    # We collect all effects here. It will be of dimension (obs_idx, response)
    mu = intercept

    # --------------------------------------------------------
    #                   DATA COMPONENTS
    # --------------------------------------------------------
    sigma = 0.5

    # Declare inputs as Data so we can change them in post-stratification
    age_id = pm.Data("age_id", age_idx, dims="obs_idx")
    a_age = hierarchical_zsn("a_age", "age", sigma=sigma)
    mu += a_age[age_id]

    education_id = pm.Data("education_id", education_idx, dims="obs_idx")
    a_edu = hierarchical_zsn("a_edu", "education", sigma=sigma)
    mu += a_edu[education_id]

    gender_id = pm.Data("gender_id", gender_idx, dims="obs_idx")
    a_gender = hierarchical_zsn("a_gender", "gender", sigma=sigma)
    mu += a_gender[gender_id]

    ethnicity_id = pm.Data("ethnicity_id", ethnicity_idx, dims="obs_idx")
    a_eth = hierarchical_zsn("a_eth", "ethnicity", sigma=sigma)
    mu += a_eth[ethnicity_id]
    
    unit_id = pm.Data("unit_id", unit_idx, dims="obs_idx")
    a_unit = hierarchical_zsn("a_unit", "unit", sigma=sigma)
    mu += a_unit[unit_id]

    ur_id = pm.Data("ur_id", ur_idx, dims="obs_idx")
    a_ur = hierarchical_zsn("a_ur", "ur", sigma=sigma)
    mu += a_ur[ur_id]


    # We can also add interaction terms:
    a_age_edu = pm.ZeroSumNormal(
        "a_age_edu",
        sigma=0.5*sigma,
        dims=("age", "education"),
        n_zerosum_axes=2,
    )
    mu += a_age_edu[age_id, education_id]

    a_gender_eth = pm.ZeroSumNormal(
        "a_gender_eth",
        sigma=0.5*sigma,
        dims=("gender", "ethnicity"),
        n_zerosum_axes=2,
    )
    mu += a_gender_eth[gender_id, ethnicity_id]

    pm.Deterministic("mu", mu, dims="obs_idx")

    # --------------------------------------------------------
    #                   OBSERVATION MODEL
    # -------------------------------------------------------- 

    # Create a mutable N and set it to 1 for now as we have one observation per person
    # This will be changed to the size of the census segment in post-stratification
    N = pm.Data("N", np.ones(len(data)), dims="obs_idx")

    # Convert the dependent variable to a dummy matrix
    obs = pd.get_dummies(data[dependent_col]).astype(int)
    model.add_coord('response',list(obs.columns))

    # Binomial observation model
    if model_type == 'binomial':
        p = pm.math.invlogit(mu)
        MvBinomial("y", p=p, N=len(obs.columns)-1, 
            n=N, observed=obs.values, dims=("obs_idx", "response"))
            
    # Ordered Multinomial observation model (Logit-based)
    elif model_type == 'ordered-logit':
        # Model for cutpoints
        cp_scale = pm.HalfNormal(f"cutpoint_sd", sigma=2)
        cp_len = len(obs.columns)-1
        
        cutpoints = pm.ZeroSumNormal(
            "cutpoints",
            sigma=cp_scale,
            transform=pm.distributions.transforms.ordered,
            initval=np.linspace(-1,1,cp_len),
            shape=cp_len
        )

        # Ordered Multinomial observation model (Logit-based)
        pm.OrderedMultinomial(
            "y",
            eta=mu,
            cutpoints=cutpoints,
            n=N,
            observed=obs.values,
            dims=("obs_idx", "response"),
        )

    # Stereotype regression
    elif model_type == 'stereotype':

        # Generate phi values: first is 0, last is 1 and the inbetween is increasing
        n_ordered = len(obs.columns)
        phi_delta = pm.Dirichlet(f'phi_diffs', [1.0]*(n_ordered-1) )
        phi = pt.concatenate([[0], pt.cumsum(phi_delta)])

        # Create baseline values
        b = (pm.Normal("stereotype_intercept", size=n_ordered, 
                                transform=pm.distributions.transforms.ZeroSumTransform([-1])))

        # Log_odds = baseline + phi*mu
        log_odds = b[None,:] + phi[None,:]*mu[:,None]
        
        # Run them through softmax
        probs = pm.math.softmax(log_odds, axis=-1)

        pm.Multinomial("y", p=probs, n=N, observed=obs.values, dims=("obs_idx", "response"))

# Do the same thing as above for post-stratification. Code in utils.py
df2, idata2 = poststratify(model,cdata,new_data)

In [ ]:
# Draw young urban female highly educated only
fquery = "gender == 'Female' and age_group2.isin(['15-19','20-29']) and education == 'Higher education'"

print(f"N in original data: {len(data.query(fquery))}")
print(f"N in synthetic population: {len(df2.query(fquery))}")

f_df = df2.query(fquery)
agg = (f_df.groupby(['response','adraw']).size()/f_df.groupby('adraw').size()).reset_index(name='p')
(ggplot(agg, aes(x='response', y='p')) +
    geom_boxplot() +
    theme(axis_text_x=element_text(angle=45, hjust=1)) +
    labs(x='Response', y='Proportion', title='Distribution of Response Proportions')
)

# Regular (non-ordered) categoricals

In [21]:
# A hierarchical building block for the model    
def hierarchical_zsn_2d(name: str, dim: str, odim = "question", sigma: float = 0.2):

    scale = pm.HalfNormal(f"{name}_sd", sigma=sigma, dims=odim) # Separate sd per output category
    offset = pm.ZeroSumNormal(f"{name}_offset", dims=(dim,odim), n_zerosum_axes=2)

    return pm.Deterministic(f"{name}", scale[None,:] * offset, dims=(dim,odim))

In [ ]:
# What are we interested in predicting?
dependent_col = 'trust2_EU' # You CAN do this with ordered, but it will be noisier

with pm.Model(coords=COORDS) as model:

    # Convert the dependent variable to a dummy matrix
    obs = pd.get_dummies(data[dependent_col]).astype(int)
    model.add_coord('response',list(obs.columns))

    # --------------------------------------------------------
    #                   Intercepts
    # --------------------------------------------------------

    # intercept (i.e national) latent popularity for each response
    intercept_sd = pm.HalfNormal("intercept_sd", 2)
    intercept = pm.Normal("intercept", sigma=intercept_sd, dims="response")

    # We collect all effects here. It will be of dimension (obs_idx, response)
    mu = intercept[None,:]

    # --------------------------------------------------------
    #                   DATA COMPONENTS
    # --------------------------------------------------------
    sigma = 0.5

    # Declare inputs as Data so we can change them in post-stratification
    age_id = pm.Data("age_id", age_idx, dims="obs_idx")
    a_age = hierarchical_zsn_2d("a_age", "age", odim="response", sigma=sigma)
    mu += a_age[age_id]

    education_id = pm.Data("education_id", education_idx, dims="obs_idx")
    a_edu = hierarchical_zsn_2d("a_edu", "education", odim="response", sigma=sigma)
    mu += a_edu[education_id]

    gender_id = pm.Data("gender_id", gender_idx, dims="obs_idx")
    a_gender = hierarchical_zsn_2d("a_gender", "gender", odim="response", sigma=sigma)
    mu += a_gender[gender_id]

    ethnicity_id = pm.Data("ethnicity_id", ethnicity_idx, dims="obs_idx")
    a_eth = hierarchical_zsn_2d("a_eth", "ethnicity", odim="response", sigma=sigma)
    mu += a_eth[ethnicity_id]
    
    unit_id = pm.Data("unit_id", unit_idx, dims="obs_idx")
    a_unit = hierarchical_zsn_2d("a_unit", "unit", odim="response", sigma=sigma)
    mu += a_unit[unit_id]

    ur_id = pm.Data("ur_id", ur_idx, dims="obs_idx")
    a_ur = hierarchical_zsn_2d("a_ur", "ur", odim="response", sigma=sigma)
    mu += a_ur[ur_id]

    # We can also add interaction terms:
    a_age_edu = pm.ZeroSumNormal(
        "a_age_edu",
        sigma=0.5*sigma,
        dims=("age", "education", "response"),
        n_zerosum_axes=3,
    )
    mu += a_age_edu[age_id, education_id]

    a_gender_eth = pm.ZeroSumNormal(
        "a_gender_eth",
        sigma=0.5*sigma,
        dims=("gender", "ethnicity", "response"),
        n_zerosum_axes=3,
    )
    mu += a_gender_eth[gender_id, ethnicity_id]

    pm.Deterministic("mu", mu, dims=("obs_idx","response"))

    # Create a mutable N and set it to 1 for now as we have one observation per person
    # This will be changed to the size of the census segment in post-stratification
    N = pm.Data("N", np.ones(len(data)), dims="obs_idx")

    # Softmax: pm version fails for some reason
    # p = pm.math.softmax(mu)
    p = pt.exp(mu)
    p/= pt.sum(p, axis=-1, keepdims=True)
    
    pm.Multinomial("y", n=N, p=p, observed=obs.values, dims=("obs_idx", "response"))

df3, idata3 = poststratify(model,cdata,new_data)